In [1]:
import os
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
import seaborn as sns

import string
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer, PorterStemmer
import re

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, accuracy_score, f1_score

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [3]:
# For handling warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\utgoy\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\utgoy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\utgoy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\utgoy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Download the data

In [6]:
train_data = pd.read_csv('../data/train.csv', header=None)
train_data.columns=['polarity', 'title', 'text',]

val_data = pd.read_csv('../data/val.csv', header=None)
val_data.columns=['polarity', 'title', 'text',]

test_data = pd.read_csv('../data/test.csv')

In [7]:
# get all train data
X_train   = train_data['text']
y_train   = train_data['polarity']

# get val data
X_val    = val_data['text']
y_val    = val_data['polarity']

# get test data
X_test   = test_data['text']
y_test   = test_data['polarity']

print(f"Train Data Shape: {X_train.shape[0]:,}")
print(f"Validation Data Shape: {X_val.shape[0]:,}")
print(f"Test Data Shape: {X_test.shape[0]:,}")

print(" ")
print(f"Number of labels = 1 in train dataset as percentage: {((y_train == 1).sum() / (X_train.shape[0])) * 100:0.2f}%")
print(f"Number of labels = 2 in train dataset as percentage: {((y_train == 2).sum() / (X_train.shape[0])) * 100:0.2f}%")

print(" ")
print(f"Number of labels = 1 in val dataset as percentage: {((y_val == 1).sum() / (X_val.shape[0])) * 100:0.2f}%")
print(f"Number of labels = 2 in val dataset as percentage: {((y_val == 2).sum() / (X_val.shape[0])) * 100:0.2f}%")

print(" ")
print(f"Number of labels = 1 in test dataset as percentage: {((y_test == 1).sum() / (X_test.shape[0])) * 100:0.2f}%")
print(f"Number of labels = 2 in test dataset as percentage: {((y_test == 2).sum() / (X_test.shape[0])) * 100:0.2f}%")

Train Data Shape: 3,600,000
Validation Data Shape: 400,000
Test Data Shape: 33,161
 
Number of labels = 1 in train dataset as percentage: 50.00%
Number of labels = 2 in train dataset as percentage: 50.00%
 
Number of labels = 1 in val dataset as percentage: 50.00%
Number of labels = 2 in val dataset as percentage: 50.00%
 
Number of labels = 1 in test dataset as percentage: 2.45%
Number of labels = 2 in test dataset as percentage: 97.55%


# Cleaning Data

## Preprocessing Helper Functions

In [10]:
def clean(text):
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', str(text), flags=re.MULTILINE)
    texter = re.sub(r"<br />", " ", text)
    texter = re.sub(r"&quot;", "\"",texter)
    texter = re.sub('&#39;', "\"", texter)
    texter = re.sub('\n', " ", texter)
    texter = re.sub(' u '," you ", texter)
    texter = re.sub('`',"", texter)
    texter = re.sub(' +', ' ', texter)
    texter = re.sub(r"(!)\1+", r"!", texter)
    texter = re.sub(r"(\?)\1+", r"?", texter)
    texter = re.sub('&amp;', 'and', texter)
    texter = re.sub('\r', ' ',texter)
    #added substitutions

    #***********added substitutions***********
    # remove all the special characters
    texter = re.sub(r'\W', ' ', texter)
    # remove all single characters
    texter = re.sub(r'\s+[a-zA-Z]\s+', ' ', texter)
    # Remove single characters from the start
    texter = re.sub(r'\^[a-zA-Z]\s+', ' ', texter)
    # Remove numbers
    texter = re.sub(r'\d+', ' ', texter)
    # Converting to Lowercase
    texter = texter.lower()
    # Remove punctuation
    texter = re.sub(r'[^\w\s]', ' ', texter)
    # Remove parentheses
    texter = re.sub(r'\([^)]*\)', ' ', texter)
    # Remove single quotes
    texter = re.sub(r'\'', ' ', texter)
    # Substituting multiple spaces with single space
    texter = re.sub(r'\s+', ' ', texter, flags=re.I)

    clean = re.compile('<.*?>')
    texter = texter.encode('ascii', 'ignore').decode('ascii')
    texter = re.sub(clean, '', texter)
    if texter == "":
        texter = ""
    return texter

def clean_dataset(dataset):
    for row in range(dataset.shape[0]):
        dataset[row,0] = clean(dataset[row,0])
    return dataset

def tokenize_lexicon(texts):
    return_texts = []
    for i in range(len(texts)):
        return_texts.append(nltk.word_tokenize(texts[i]))
        return_texts[i] = nltk.pos_tag(return_texts[i])
    return return_texts

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wn.ADJ
    elif pos_tag.startswith('V'):
        return wn.VERB
    elif pos_tag.startswith('N'):
        return wn.NOUN
    elif pos_tag.startswith('R'):
        return wn.ADV
    else:
        return wn.NOUN

def lemmatize_texts(texts):
    return_texts = []
    lemmer = nltk.stem.WordNetLemmatizer()
    for i in range(len(texts)):
        return_texts.append([])
        for j in range(len(texts[i])):
                return_texts[i].append(lemmer.lemmatize(texts[i][j][0], pos=get_wordnet_pos(texts[i][j][1])))
    return return_texts

def stem_texts(texts):
    return_texts = []
    ps = PorterStemmer()
    for i in range(len(texts)):
        return_texts.append([])
        for j in range(len(texts[i])):
                return_texts[i].append(ps.stem(texts[i][j][0]))
    return return_texts


def backtostring(texts):
    return_texts = []
    for i in range(len(texts)):
        return_texts.append(" ".join(texts[i]))
    return return_texts

In [11]:
def pre_process(data):
    preproc_data = data.copy()
    preproc_data = preproc_data.str.lower()

    mapping = str.maketrans("", "", string.punctuation)
    preproc_data = preproc_data.str.translate(mapping)
    
    stop_words = set(stopwords.words('english'))
    preproc_data = preproc_data.apply(lambda text: ' '.join([word for word in str(text).split() if word.lower() not in stop_words]))
       
    preproc_data = preproc_data.apply(lambda text: re.sub(r'@\w+', '', re.sub(r'http\S+|www\S+', '', text)))
    return preproc_data

In [12]:
# get the preprocessed data
X_train_preproc = pre_process(X_train)
X_val_preproc   = pre_process(X_val)
X_test_preproc  = pre_process(X_test)

# Vectorization: TF-IDF

In [14]:
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [15]:
tfidf = TfidfVectorizer(strip_accents='unicode', lowercase=True, tokenizer=tokenizer_porter, stop_words='english', max_df=0.9)
X_train_tfidf = tfidf.fit_transform(X_train_preproc)
print(f"\nTF-IDF feature matrix shape: {X_train_tfidf.shape}")


TF-IDF feature matrix shape: (3600000, 1956697)


In [16]:
X_val_tfidf = tfidf.transform(X_val_preproc)
print(f"\nTF-IDF feature matrix shape: {X_val_tfidf.shape}")


TF-IDF feature matrix shape: (400000, 1956697)


In [17]:
X_test_tfidf = tfidf.transform(X_test_preproc)
print(f"\nTF-IDF feature matrix shape: {X_test_tfidf.shape}")


TF-IDF feature matrix shape: (33161, 1956697)


# Models

## Logistic Regression

In [44]:
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train_tfidf, y_train)

LogisticRegression(C=5, max_iter=1000, random_state=42)

In [45]:
# Validate on validation set
y_val_pred_lr = lr.predict(X_val_tfidf)
val_accuracy_lr = lr.score(X_val_tfidf, y_val)

In [46]:
print("\n--- Logistic Regression ---")
print(f"Validation Accuracy: {val_accuracy_lr:.4f}")
print(f"Validation F1 Score: {f1_score(y_val, y_val_pred_lr, average='weighted'):.4f}")
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred_lr))


--- Logistic Regression ---
Validation Accuracy: 0.8629
Validation F1 Score: 0.8629
Validation Classification Report:
              precision    recall  f1-score   support

           1       0.87      0.86      0.86    200000
           2       0.86      0.87      0.86    200000

    accuracy                           0.86    400000
   macro avg       0.86      0.86      0.86    400000
weighted avg       0.86      0.86      0.86    400000



In [47]:
# Validate on validation set
y_test_pred_lr = lr.predict(X_test_tfidf)
test_accuracy_lr = lr.score(X_test_tfidf, y_test)

In [48]:
print("\n--- Logistic Regression ---")
print(f"Validation Accuracy: {test_accuracy_lr:.4f}")
print(f"Validation F1 Score: {f1_score(y_test, y_test_pred_lr, average='weighted'):.4f}")
print("Validation Classification Report:")
print(classification_report(y_test, y_test_pred_lr))


--- Logistic Regression ---
Validation Accuracy: 0.9055
Validation F1 Score: 0.9334
Validation Classification Report:
              precision    recall  f1-score   support

           1       0.18      0.81      0.30       812
           2       0.99      0.91      0.95     32349

    accuracy                           0.91     33161
   macro avg       0.59      0.86      0.62     33161
weighted avg       0.97      0.91      0.93     33161



## MultinomalNB

## KKN

## Random Forest Classifier

## XGBoost

## LightGBM

# Hyperparameter Tuning